# Import

In [1]:
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.6")' SwiftCoreMLTools
%install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

Installing packages:
	.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.6")
		SwiftCoreMLTools
	.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")
		Just
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpwxxl_7ss/swift-install
Fetching https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Fetching https://github.com/dduan/Just.git
Fetching https://github.com/apple/swift-protobuf.git
Cloning https://github.com/dduan/Just.git
Resolving https://github.com/dduan/Just.git at 0.8.0
Cloning https://github.com/apple/swift-protobuf.git
Resolving https://github.com/apple/swift-protobuf.git at 1.8.0
Cloning https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Resolving https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git at 0.0.6
[1/3] Compiling Just Just.swift
[2/3] Compiling SwiftProtobuf AnyMessageStorage.swift
[3/4] Compiling SwiftCoreMLTools Activations.swift
[4/5] Compiling jupyterInstalledPackages jupyterInstalled

In [2]:
import Foundation
import SwiftCoreMLTools
import TensorFlow
import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per ten thousand dollars
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in a thousand dollar

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [3]:
// if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
//     try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
// }

# Data Ingestion

In [4]:
let data = try String(contentsOfFile:"./data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

let dataFeatures = dataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = dataRecords.map{ Array($0[(numColumns-1)...]) }

# Data Transformation

## Split Numerical Categorical Features

In [5]:
let categoricalColumns = [3, 8]
let numericalColumns = [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12]
let numCategoricalFeatures = categoricalColumns.count
let numNumericalFeatures = numericalColumns.count
let numLabels = 1

assert(numColumns == numCategoricalFeatures + numNumericalFeatures + 1)

// Get Categorical Features
let allCategoriesValues = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
                                .reduce(into: Array(repeating: [Int32](), count: 2)){ total, value in
                                    total[0].append(value[0])
                                    total[1].append(value[1]) }
                                .map{ Set($0).sorted() }

let categoricalFeatures = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }

// Get Numerical Features
let numericalFeatures = dataFeatures.map{ row in numericalColumns.map{ row[$0] } }

## Categorize Categorical Features with Ordinal values

In [6]:
var categoricalValues = Array(repeating: Set<Int32>(), count: 2)

for record in categoricalFeatures {
    categoricalValues[0].insert(record[0])
    categoricalValues[1].insert(record[1])
}

let sortedCategoricalValues = [categoricalValues[0].sorted(), categoricalValues[1].sorted()]

let ordinalCategoricalFeatures = categoricalFeatures.map{ [Int32(sortedCategoricalValues[0].firstIndex(of:$0[0])!), 
                                                           Int32(sortedCategoricalValues[1].firstIndex(of:$0[1])!)] }

## Split Train and Test

In [7]:
let trainPercentage:Float = 0.8
let numTrainRecords = Int(ceil(Float(numRecords) * trainPercentage))
let numTestRecords = numRecords - numTrainRecords

func matrixTranspose<T>(_ matrix: [[T]]) -> [[T]] {
    if matrix.isEmpty {return matrix}
    var result = [[T]]()
    for index in 0..<matrix.first!.count {
        result.append(matrix.map{$0[index]})
    }
    return result
}

let xCategoricalAllTrain = matrixTranspose(Array(ordinalCategoricalFeatures[0..<numTrainRecords]))
let xCategoricalAllTest = matrixTranspose(Array(ordinalCategoricalFeatures[numTrainRecords...]))
let xNumericalAllTrain = Array(Array(numericalFeatures[0..<numTrainRecords]).joined())
let xNumericalAllTest = Array(Array(numericalFeatures[numTrainRecords...]).joined())
let yAllTrain = Array(Array(dataLabels[0..<numTrainRecords]).joined())
let yAllTest = Array(Array(dataLabels[numTrainRecords...]).joined())

let XCategoricalTrain = xCategoricalAllTrain.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTrainRecords, 1]))
}
let XCategoricalTest = xCategoricalAllTest.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTestRecords, 1]))
}

let XNumericalTrainDeNorm = Tensor<Float>(xNumericalAllTrain).reshaped(to: TensorShape([numTrainRecords, numNumericalFeatures]))
let XNumericalTestDeNorm = Tensor<Float>(xNumericalAllTest).reshaped(to: TensorShape([numTestRecords, numNumericalFeatures]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, numLabels]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, numLabels]))

## Normalize Numerical Features

In [8]:
let mean = XNumericalTrainDeNorm.mean(alongAxes: 0)
let std = XNumericalTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

[[ 2.0137098,  14.197531,   9.523555, 0.53213036,  6.3311296,   64.47929,  4.1678762,  353.68396,
    18.03163,  379.84735,  11.394517]] [[ 6.5076075,  25.258776,   6.534038, 0.11449408,  0.7311985,  29.000755,  2.1797554,  132.14561,
    2.217345,  40.494495,   6.852825]]


In [9]:
let XNumericalTrain = (XNumericalTrainDeNorm - mean)/std
let XNumericalTest = (XNumericalTestDeNorm - mean)/std

In [10]:
print("Training shapes \(XNumericalTrain.shape) \(XCategoricalTrain[0].shape) \(XCategoricalTrain[1].shape) \(YTrain.shape)")
print("Testing shapes  \(XNumericalTest.shape) \(XCategoricalTest[0].shape) \(XCategoricalTest[1].shape) \(YTest.shape)")

Training shapes [405, 11] [405, 1] [405, 1] [405, 1]
Testing shapes  [101, 11] [101, 1] [101, 1] [101, 1]


# Model

In [11]:
struct MultiInputs<N: Differentiable, C>: Differentiable {
  var numerical: N
  
  @noDerivative
  var categorical: C

  @differentiable
  init(numerical: N, categorical: C) {
    self.numerical = numerical
    self.categorical = categorical
  }
}

struct RegressionModel: Module {
    var embedding1 = TensorFlow.Embedding<Float>(vocabularySize: 2, embeddingSize: 2)
    var embedding2 = TensorFlow.Embedding<Float>(vocabularySize: 9, embeddingSize: 5)
    var allInputConcatLayer = Dense<Float>(inputSize: (11 + 2 + 5), outputSize: 64, activation: relu)
    var hiddenLayer = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var outputLayer = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: MultiInputs<[Tensor<Float>], [Tensor<Int32>]>) -> Tensor<Float> {
        let embeddingOutput1 = embedding1(input.categorical[0])
        let embeddingOutput1Reshaped = embeddingOutput1.reshaped(to: 
            TensorShape([embeddingOutput1.shape[0], embeddingOutput1.shape[2]]))
        let embeddingOutput2 = embedding2(input.categorical[1])
        let embeddingOutput2Reshaped = embeddingOutput2.reshaped(to: 
            TensorShape([embeddingOutput2.shape[0], embeddingOutput2.shape[2]]))
        let allConcat = Tensor<Float>(concatenating: [input.numerical[0], embeddingOutput1Reshaped, embeddingOutput2Reshaped], alongAxis: 1)
        return allConcat.sequenced(through: allInputConcatLayer, hiddenLayer, outputLayer)
    }
}

var model = RegressionModel()

# Training

In [12]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [13]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(numTrainRecords) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [14]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(numTrainRecords, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                         categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                       XCategoricalTrain[1][batchStart..<batchEnd]])
            let logits = model(multiInput)
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                     categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                   XCategoricalTrain[1][batchStart..<batchEnd]])
        let logits = model(multiInput)
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 629.58466, MAE: 23.11697
Epoch 2: MSE: 569.8215, MAE: 21.903145
Epoch 3: MSE: 507.82208, MAE: 20.533684
Epoch 4: MSE: 437.7764, MAE: 18.820034
Epoch 5: MSE: 362.32455, MAE: 16.728703
Epoch 6: MSE: 281.21158, MAE: 14.396033
Epoch 7: MSE: 203.11066, MAE: 11.521348
Epoch 8: MSE: 139.76155, MAE: 8.793625
Epoch 9: MSE: 97.55573, MAE: 6.717092
Epoch 10: MSE: 72.564285, MAE: 5.4857526
Epoch 11: MSE: 56.397984, MAE: 4.7238894
Epoch 12: MSE: 48.320656, MAE: 4.3293858
Epoch 13: MSE: 43.571487, MAE: 4.110623
Epoch 14: MSE: 40.232944, MAE: 3.8525772
Epoch 15: MSE: 37.14219, MAE: 3.750439
Epoch 16: MSE: 35.128788, MAE: 3.5926704
Epoch 17: MSE: 33.282864, MAE: 3.4505336
Epoch 18: MSE: 31.539206, MAE: 3.3956962
Epoch 19: MSE: 30.507399, MAE: 3.2626429
Epoch 20: MSE: 29.406546, MAE: 3.2253046
Epoch 21: MSE: 28.389687, MAE: 3.1390784
Epoch 22: MSE: 27.679405, MAE: 3.0726068
Epoch 23: MSE: 26.825886, MAE: 3.0767207
Epoch 24: MSE: 26.078176, MAE: 3.0019925
Epoch 25: MSE: 25.744408, MAE: 2.9

Epoch 204: MSE: 4.5593343, MAE: 1.3742963
Epoch 205: MSE: 4.6487865, MAE: 1.3793479
Epoch 206: MSE: 4.414052, MAE: 1.3822147
Epoch 207: MSE: 4.7290735, MAE: 1.3812848
Epoch 208: MSE: 4.1453533, MAE: 1.3496896
Epoch 209: MSE: 4.7030582, MAE: 1.375815
Epoch 210: MSE: 4.4231906, MAE: 1.3613276
Epoch 211: MSE: 4.3875513, MAE: 1.3606963
Epoch 212: MSE: 4.113841, MAE: 1.3427184
Epoch 213: MSE: 4.3402133, MAE: 1.362984
Epoch 214: MSE: 4.3597045, MAE: 1.3436236
Epoch 215: MSE: 4.299067, MAE: 1.3543054
Epoch 216: MSE: 4.294114, MAE: 1.3368479
Epoch 217: MSE: 4.216857, MAE: 1.3340079
Epoch 218: MSE: 4.601817, MAE: 1.3581208
Epoch 219: MSE: 4.0256743, MAE: 1.337351
Epoch 220: MSE: 4.1342416, MAE: 1.3337433
Epoch 221: MSE: 4.058764, MAE: 1.3229325
Epoch 222: MSE: 4.1086392, MAE: 1.3208662
Epoch 223: MSE: 4.1123614, MAE: 1.3081975
Epoch 224: MSE: 4.070235, MAE: 1.3242009
Epoch 225: MSE: 4.0501947, MAE: 1.317912
Epoch 226: MSE: 3.971472, MAE: 1.2927272
Epoch 227: MSE: 4.0339823, MAE: 1.2983512
Epoch

Epoch 402: MSE: 2.6133566, MAE: 1.0423671
Epoch 403: MSE: 2.406435, MAE: 1.050483
Epoch 404: MSE: 2.3432908, MAE: 1.1001995
Epoch 405: MSE: 2.6220045, MAE: 1.0788184
Epoch 406: MSE: 2.4184482, MAE: 1.0323774
Epoch 407: MSE: 2.6634583, MAE: 1.0649166
Epoch 408: MSE: 2.455541, MAE: 0.9939105
Epoch 409: MSE: 2.5038664, MAE: 1.069643
Epoch 410: MSE: 2.338223, MAE: 1.0484989
Epoch 411: MSE: 2.4772298, MAE: 1.0958463
Epoch 412: MSE: 2.4899092, MAE: 1.0576552
Epoch 413: MSE: 2.2908878, MAE: 1.0070841
Epoch 414: MSE: 2.462776, MAE: 1.0786923
Epoch 415: MSE: 2.4756677, MAE: 1.0475224
Epoch 416: MSE: 2.1973124, MAE: 1.0063245
Epoch 417: MSE: 2.2664263, MAE: 1.0355222
Epoch 418: MSE: 2.3486886, MAE: 1.045028
Epoch 419: MSE: 2.6732073, MAE: 1.030545
Epoch 420: MSE: 2.401425, MAE: 0.985335
Epoch 421: MSE: 2.3671148, MAE: 1.0460799
Epoch 422: MSE: 2.524806, MAE: 1.0412619
Epoch 423: MSE: 2.2683346, MAE: 0.9877845
Epoch 424: MSE: 2.4685006, MAE: 1.0427268
Epoch 425: MSE: 2.2109466, MAE: 1.0170729
Epo

# Test

In [15]:
Context.local.learningPhase = .inference

let multiInputTest = MultiInputs(numerical: [XNumericalTest],
                                 categorical: [XCategoricalTest[0],
                                               XCategoricalTest[1]])

let prediction = model(multiInputTest)

let predictionMse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numTestRecords)
let predictionMae = mae(predictions: prediction, truths: YTest)/Float(numTestRecords)

print("MSE: \(predictionMse), MAE: \(predictionMae)")

MSE: 0.5485814, MAE: 0.052630015


# Export

In [16]:
print(model.embedding1.embeddings.shape, model.embedding2.embeddings.shape)

[2, 2] [9, 5]


In [17]:
let coremlModel = Model(version: 4,
                        shortDescription: "Regression",
                        author: "Jacopo Mangiavacchi",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.6"]) {
    Input(name: "numericalInput", shape: [11])
    Input(name: "categoricalInput1", shape: [1])
    Input(name: "categoricalInput2", shape: [1])
    Output(name: "output", shape: [1])
    NeuralNetwork {
        Embedding(name: "embedding1",
                     input: ["categoricalInput1"],
                     output: ["outEmbedding1"],
                     weight: model.embedding1.embeddings.transposed().flattened().scalars,
                     inputDim: 2,
                     outputChannels: 2)
        Permute(name: "permute1",
                     input: ["outEmbedding1"],
                     output: ["outPermute1"],
                     axis: [2, 1, 0, 3])
        Flatten(name: "flatten1",
                     input: ["outPermute1"],
                     output: ["outFlatten1"],
                     mode: .last)
        Embedding(name: "embedding2",
                     input: ["categoricalInput2"],
                     output: ["outEmbedding2"],
                     weight: model.embedding2.embeddings.transposed().flattened().scalars,
                     inputDim: 9,
                     outputChannels: 5)
        Permute(name: "permute2",
                     input: ["outEmbedding2"],
                     output: ["outPermute2"],
                     axis: [2, 1, 0, 3])
        Flatten(name: "flatten2",
                     input: ["outPermute2"],
                     output: ["outFlatten2"],
                     mode: .last)
        Concat(name: "concat",
                     input: ["numericalInput", "outFlatten1", "outFlatten2"],
                     output: ["outConcat"])
        InnerProduct(name: "dense1",
                     input: ["outConcat"],
                     output: ["outDense1"],
                     weight: model.allInputConcatLayer.weight.transposed().flattened().scalars,
                     bias: model.allInputConcatLayer.bias.flattened().scalars,
                     inputChannels: 11 + 2 + 5,
                     outputChannels: 64)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     weight: model.hiddenLayer.weight.transposed().flattened().scalars,
                     bias: model.hiddenLayer.bias.flattened().scalars,
                     inputChannels: 64,
                     outputChannels: 32)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     weight: model.outputLayer.weight.transposed().flattened().scalars,
                     bias: model.outputLayer.bias.flattened().scalars,
                     inputChannels: 32,
                     outputChannels: 1)
    }
}

In [18]:
let coreMLData = coremlModel.coreMLData
try! coreMLData!.write(to: URL(fileURLWithPath: "./s4tf_house_simplified_trained_model.mlmodel"))

In [19]:
print(XNumericalTest[0], XCategoricalTest[0][0], XCategoricalTest[1][0], "=>", YTest[0])
print(XNumericalTest[17], XCategoricalTest[0][17], XCategoricalTest[1][17], "=>", YTest[17])
print(XNumericalTest[87], XCategoricalTest[0][87], XCategoricalTest[1][87], "=>", YTest[87])

[  10.127699, -0.56208307,   1.3125796,   1.4050479,  -0.8863933,   1.2248203,  -1.2581577,
   2.3634233,   0.9779133,  0.12650238,   1.6906141] [0] [8] => [5.0]
[   1.541963, -0.56208307,   1.3125796,   0.7150559, -0.93426037,   0.7972452,  -1.0169381,
   2.3634233,   0.9779133,   -2.180478,  0.39479825] [0] [8] => [20.8]
[-0.29233322, -0.56208307,   2.7879307,   0.6713855,  -0.4761084,   0.6558696, -0.94413173,
   2.7039568,   0.9328142,  0.42111015,  0.28535435] [0] [3] => [20.1]
